In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
%%nuclio cmd
pip install kfserving
pip install numpy
pip install xgboost

    100% |████████████████████████████████| 81kB 3.4MB/s ta 0:00:011
    100% |████████████████████████████████| 1.5MB 20.7MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 29.4MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 31.9MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 39.1MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 25.5MB/s ta 0:00:01
kfp 0.1.30 has requirement kubernetes<=9.0.0,>=8.0.0, but you'll have kubernetes 10.0.1 which is incompatible.
  Found existing installation: kubernetes 9.0.0
    Uninstalling kubernetes-9.0.0:
      Successfully uninstalled kubernetes-9.0.0


In [2]:
import kfserving
import os
import numpy as np
from typing import List, Any

import xgboost as xgb

In [3]:
BOOSTER_FILE = "model.bst"

class XGBoostModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, booster: xgb.XGBModel = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not booster is None:
            self._booster = booster
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), BOOSTER_FILE)
        self._booster = xgb.Booster(model_file=model_file)
        self.ready = True

    def predict(self, body: List) -> List:
        try:
            # Use of list as input is deprecated see https://github.com/dmlc/xgboost/pull/3970
            dmatrix = xgb.DMatrix(body)
            result: xgb.DMatrix = self._booster.predict(dmatrix)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)


In [4]:
# nuclio: end-code
from mlrun import new_model_server
import requests

In [5]:
fn = new_model_server('mysrv2', models={'netops_v1': 'file:///User/mlrun'}, model_class='XGBoostModel')
fn.with_v3io('User','~/') 

In [6]:
addr = fn.deploy(project='xgb')

[nuclio.deploy] 2019-09-30 23:14:23,315 (info) Building processor image
[nuclio.deploy] 2019-09-30 23:14:28,355 (info) Build complete
[nuclio.deploy] 2019-09-30 23:14:32,391 (info) Function deploy complete
[nuclio.deploy] 2019-09-30 23:14:32,396 done updating mysrv2, function address: 3.15.147.113:30953


In [7]:
# Seldon protocol event
event_seldon = {"data": {"ndarray": [[5], [10]]}}
csel = str(event_seldon).replace("\'", "\"")

In [8]:
resp = requests.put(addr + '/predict/netops_v1', data=csel)
print(resp.content)

b'{"data": {"ndarray": [[0.9755825400352478, 0.005246310960501432, 0.0076179164461791515, 0.001650458201766014, 0.001650458201766014, 0.001650458201766014, 0.001650458201766014, 0.001650458201766014, 0.001650458201766014, 0.001650458201766014], [0.9707112312316895, 0.010213389992713928, 0.0075798784382641315, 0.0016422170447185636, 0.0016422170447185636, 0.0016422170447185636, 0.0016422170447185636, 0.0016422170447185636, 0.0016422170447185636, 0.0016422170447185636]]}}'
